In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')

In [2]:
np.random.seed(51)

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
%%time
train_metadata_kaggle = pd.read_csv('train_metadata_kaggle.csv')
test_metadata_kaggle = pd.read_csv('test_metadata_kaggle.csv')

CPU times: user 49.1 s, sys: 2.14 s, total: 51.3 s
Wall time: 51.3 s


In [5]:
test_id = test_metadata_kaggle['object_id']

In [6]:
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)

(7848, 51) (3492890, 50)


In [7]:
test_metadata_kaggle.head()

,object_id,flux_min,flux_max,flux_mean,flux_median,flux_std,flux_skew,flux_err_min,flux_err_max,flux_err_mean,flux_err_median,flux_err_std,flux_err_skew,detected_mean,flux_ratio_sq_sum,flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_by_flux_ratio_sq_skew,flux_diff,flux_dif2,flux_w_mean,flux_dif3,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__kurtosis,0__skewness,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__kurtosis,1__skewness,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",2__kurtosis,2__skewness,"3__fft_coefficient__coeff_0__attr_""abs""","3__fft_coefficient__coeff_1__attr_""abs""",3__kurtosis,3__skewness,"4__fft_coefficient__coeff_0__attr_""abs""","4__fft_coefficient__coeff_1__attr_""abs""",4__kurtosis,4__skewness,"5__fft_coefficient__coeff_0__attr_""abs""","5__fft_coefficient__coeff_1__attr_""abs""",5__kurtosis,5__skewness,mjd_diff_det,hostgal_photoz,hostgal_photoz_err,mwebv
0,13,-12.680235,42.765503,3.997127,0.616561,9.149645,2.037355,0.691634,11.257108,2.461810,1.972973,1.718101,1.826388,0.157576,7806.412424,4.771625,1.896346e+05,5.396523,55.445738,13.871398,24.292155,2.282455,29.002872,37.684425,-0.247160,0.147622,92.039690,79.990817,10.567412,3.117684,164.640622,139.733762,4.512783,2.171455,300.547278,246.788411,2.951479,1.806797,408.305525,349.008202,0.094963,1.153076,324.515880,251.116361,0.072799,0.804929,119.8531,0.3193,0.0542,0.019
1,14,-11.142164,14.839427,0.884047,0.072856,3.399946,0.970525,0.690589,11.249375,2.457580,1.973559,1.717591,1.826703,0.012121,806.406927,11.486148,5.525817e+03,12.348124,25.981591,29.389389,6.852393,3.791608,22.708482,26.159787,0.099267,0.502325,13.685195,27.630359,13.429229,3.109318,31.012899,33.427074,13.769006,3.247873,56.042403,59.784625,1.750840,1.082798,83.561278,77.494564,-0.322108,-0.099957,84.725142,27.013154,-0.348744,0.132025,28.0053,0.6323,0.0179,0.018
2,17,-14.202744,16.761280,0.791032,0.458390,3.886578,0.377131,0.663680,11.278636,2.702947,2.184483,1.922641,1.802497,0.014205,784.835502,9.509911,4.124400e+03,9.923556,30.964024,39.143819,5.255113,5.892170,6.030862,46.450439,0.590786,-0.427429,53.366119,41.192678,4.127763,1.710048,58.607170,42.353288,8.642889,2.570936,26.975615,59.958041,3.484929,1.141959,57.222812,72.448018,2.085327,0.429582,76.240782,73.533783,0.063837,-0.175525,2.7628,0.8297,0.0605,0.016
3,23,-12.631923,28.061138,0.970396,0.465986,4.100713,2.177402,0.697639,11.305429,2.480364,2.003704,1.723367,1.821626,0.018182,876.027511,8.010597,8.293673e+03,9.227223,40.693061,41.934474,9.467365,4.298245,0.776942,14.926490,-0.580529,-0.058688,2.371129,22.476077,1.429683,-0.000571,34.447730,14.240274,1.223998,0.468594,58.911457,62.709899,4.269139,1.414380,115.918277,70.371095,7.377841,2.409871,112.547521,123.413344,1.793238,0.953107,3.0125,0.6533,0.1479,0.023
4,34,-13.239577,124.475609,4.580070,0.301366,19.862714,4.537527,0.679312,11.365292,2.746784,2.210577,1.930790,1.798629,0.065341,47612.580669,9.140568,4.815012e+06,10.251332,137.715186,30.068359,101.128982,1.361778,65.064185,55.191226,7.457425,2.135457,202.641918,191.410916,16.393710,4.133929,425.940451,414.455381,15.206929,4.036551,368.228927,360.630612,15.386561,4.030676,377.744797,301.983950,15.302549,3.992740,172.564336,289.757599,12.507289,3.513977,25.0572,0.4617,0.0122,0.023


In [8]:
%%time
train_metadata = pd.read_csv('train_metadata_final.csv')
test_metadata = pd.read_csv('test_metadata_final.csv')

CPU times: user 1min 44s, sys: 6.69 s, total: 1min 51s
Wall time: 1min 51s


In [9]:
y = train_metadata['target']
del train_metadata['target']
gc.collect()

2401

In [10]:
will_be_deleted_columns = ['NG_min_flux','NG_max_flux','NG_std_flux','NG_mean_flux','NG_median_flux',
                          'NG_min_flux_err','NG_max_flux_err','NG_std_flux_err','NG_mean_flux_err','NG_median_flux_err',
                          'NG_mean_detected','hostgal_photoz','hostgal_photoz_err','mwebv']

In [11]:
%%time
train_metadata.drop(will_be_deleted_columns,inplace=True , axis=1)
test_metadata.drop(will_be_deleted_columns,inplace=True , axis=1)
gc.collect()

CPU times: user 1.53 s, sys: 2.42 s, total: 3.95 s
Wall time: 3.91 s


In [12]:
print(train_metadata.shape,test_metadata.shape)

(7848, 218) (3492890, 218)


In [13]:
test_metadata.head()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,distmod,NG_sum_flux_err,NG_sum_flux,NG_count_detected,NG_std_detected,NG_sum_detected,NG_median_detected,A0_min_flux_err,A0_max_flux_err,A0_std_flux_err,A0_sum_flux_err,A0_mean_flux_err,A0_median_flux_err,A0_min_flux,A0_max_flux,A0_std_flux,A0_sum_flux,A0_mean_flux,A0_median_flux,A0_count_detected,A0_std_detected,A0_sum_detected,A0_mean_detected,A0_median_detected,A1_min_flux_err,A1_max_flux_err,A1_std_flux_err,A1_sum_flux_err,A1_mean_flux_err,A1_median_flux_err,A1_min_flux,A1_max_flux,A1_std_flux,A1_sum_flux,A1_mean_flux,A1_median_flux,A1_count_detected,A1_std_detected,A1_sum_detected,A1_mean_detected,A1_median_detected,A2_min_flux_err,A2_max_flux_err,A2_std_flux_err,A2_sum_flux_err,A2_mean_flux_err,A2_median_flux_err,A2_min_flux,A2_max_flux,A2_std_flux,A2_sum_flux,A2_mean_flux,A2_median_flux,A2_count_detected,A2_std_detected,A2_sum_detected,A2_mean_detected,A2_median_detected,A3_min_flux_err,A3_max_flux_err,A3_std_flux_err,A3_sum_flux_err,A3_mean_flux_err,A3_median_flux_err,A3_min_flux,A3_max_flux,A3_std_flux,A3_sum_flux,A3_mean_flux,A3_median_flux,A3_count_detected,A3_std_detected,A3_sum_detected,A3_mean_detected,A3_median_detected,A4_min_flux_err,A4_max_flux_err,A4_std_flux_err,A4_sum_flux_err,A4_mean_flux_err,A4_median_flux_err,A4_min_flux,A4_max_flux,A4_std_flux,A4_sum_flux,A4_mean_flux,A4_median_flux,A4_count_detected,A4_std_detected,A4_sum_detected,A4_mean_detected,A4_median_detected,A5_min_flux_err,A5_max_flux_err,A5_std_flux_err,A5_sum_flux_err,A5_mean_flux_err,A5_median_flux_err,A5_min_flux,A5_max_flux,A5_std_flux,A5_sum_flux,A5_mean_flux,A5_median_flux,A5_count_detected,A5_std_detected,A5_sum_detected,A5_mean_detected,A5_median_detected,p0_region_minus_4,p0_region_minus_3,p0_region_minus_2,p0_region_minus_1,p0_region_plus_1,p0_region_plus_2,p0_region_plus_3,p0_region_plus_4,p1_region_minus_4,p1_region_minus_3,p1_region_minus_2,p1_region_minus_1,p1_region_plus_1,p1_region_plus_2,p1_region_plus_3,p1_region_plus_4,p2_region_minus_4,p2_region_minus_3,p2_region_minus_2,p2_region_minus_1,p2_region_plus_1,p2_region_plus_2,p2_region_plus_3,p2_region_plus_4,p3_region_minus_4,p3_region_minus_3,p3_region_minus_2,p3_region_minus_1,p3_region_plus_1,p3_region_plus_2,p3_region_plus_3,p3_region_plus_4,p4_region_minus_4,p4_region_minus_3,p4_region_minus_2,p4_region_minus_1,p4_region_plus_1,p4_region_plus_2,p4_region_plus_3,p4_region_plus_4,p5_region_minus_4,p5_region_minus_3,p5_region_minus_2,p5_region_minus_1,p5_region_plus_1,p5_region_plus_2,p5_region_plus_3,p5_region_plus_4,number_of_0s,number_of_1s,number_of_2s,number_of_3s,number_of_4s,number_of_5s,percent_p0_region_minus_4,percent_p0_region_minus_3,percent_p0_region_minus_2,percent_p0_region_minus_1,percent_p0_region_plus_1,percent_p0_region_plus_2,percent_p0_region_plus_3,percent_p0_region_plus_4,percent_p1_region_minus_4,percent_p1_region_minus_3,percent_p1_region_minus_2,percent_p1_region_minus_1,percent_p1_region_plus_1,percent_p1_region_plus_2,percent_p1_region_plus_3,percent_p1_region_plus_4,percent_p2_region_minus_4,percent_p2_region_minus_3,percent_p2_region_minus_2,percent_p2_region_minus_1,percent_p2_region_plus_1,percent_p2_region_plus_2,percent_p2_region_plus_3,percent_p2_region_plus_4,percent_p3_region_minus_4,percent_p3_region_minus_3,percent_p3_region_minus_2,percent_p3_region_minus_1,percent_p3_region_plus_1,percent_p3_region_plus_2,percent_p3_region_plus_3,percent_p3_region_plus_4,percent_p4_region_minus_4,percent_p4_region_minus_3,percent_p4_region_minus_2,percent_p4_region_minus_1,percent_p4_region_plus_1,percent_p4_region_plus_2,percent_p4_region_plus_3,percent_p4_region_plus_4,percent_p5_region_minus_4,percent_p5_region_minus_3,percent_p5_region_minus_2,percent_p5_region_minus_1,percent_p5_region_plus_1,percent_p5_region_plus_2,percent_p5_region_plus_3,percent_p5_region_plus_4
0,13,34.44,-5.230,170.0,-59.97,1,0.3047,41.12,812.39730,1319.05190,330.0,0.3650,52.0,0.0,1.122888,3.087183,0.424485,146.99579,2.0416

In [14]:
#for i,j in enumerate(list(train_metadata.columns.values)):
#    print(i,j)

In [15]:
NG_columns = [x for x in list(train_metadata.columns.values) if x[:2] =="NG"]

In [16]:
A0_columns = [x for x in list(train_metadata.columns.values) if x[:2] =="A0"]

In [17]:
NG_columns

['NG_sum_flux_err',
 'NG_sum_flux',
 'NG_count_detected',
 'NG_std_detected',
 'NG_sum_detected',
 'NG_median_detected']

In [18]:
ignored_columns = ['ra','decl','gal_l','gal_b','ddf','hostgal_specz','distmod']

In [19]:
temp = train_metadata_kaggle.merge(right = train_metadata[[x for x in train_metadata.columns if x not in ignored_columns ]] ,how='left',on = 'object_id')

In [20]:
del temp['target']

In [21]:
print(temp.shape)

(7848, 260)


In [22]:
temp.head()

,flux_min,flux_max,flux_mean,flux_median,flux_std,flux_skew,flux_err_min,flux_err_max,flux_err_mean,flux_err_median,flux_err_std,flux_err_skew,detected_mean,flux_ratio_sq_sum,flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_by_flux_ratio_sq_skew,flux_diff,flux_dif2,flux_w_mean,flux_dif3,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__kurtosis,0__skewness,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__kurtosis,1__skewness,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",2__kurtosis,2__skewness,"3__fft_coefficient__coeff_0__attr_""abs""","3__fft_coefficient__coeff_1__attr_""abs""",3__kurtosis,3__skewness,"4__fft_coefficient__coeff_0__attr_""abs""","4__fft_coefficient__coeff_1__attr_""abs""",4__kurtosis,4__skewness,"5__fft_coefficient__coeff_0__attr_""abs""","5__fft_coefficient__coeff_1__attr_""abs""",5__kurtosis,5__skewness,mjd_diff_det,hostgal_photoz,hostgal_photoz_err,mwebv,object_id,NG_sum_flux_err,NG_sum_flux,NG_count_detected,NG_std_detected,NG_sum_detected,NG_median_detected,A0_min_flux_err,A0_max_flux_err,A0_std_flux_err,A0_sum_flux_err,A0_mean_flux_err,A0_median_flux_err,A0_min_flux,A0_max_flux,A0_std_flux,A0_sum_flux,A0_mean_flux,A0_median_flux,A0_count_detected,A0_std_detected,A0_sum_detected,A0_mean_detected,A0_median_detected,A1_min_flux_err,A1_max_flux_err,A1_std_flux_err,A1_sum_flux_err,A1_mean_flux_err,A1_median_flux_err,A1_min_flux,A1_max_flux,A1_std_flux,A1_sum_flux,A1_mean_flux,A1_median_flux,A1_count_detected,A1_std_detected,A1_sum_detected,A1_mean_detected,A1_median_detected,A2_min_flux_err,A2_max_flux_err,A2_std_flux_err,A2_sum_flux_err,A2_mean_flux_err,A2_median_flux_err,A2_min_flux,A2_max_flux,A2_std_flux,A2_sum_flux,A2_mean_flux,A2_median_flux,A2_count_detected,A2_std_detected,A2_sum_detected,A2_mean_detected,A2_median_detected,A3_min_flux_err,A3_max_flux_err,A3_std_flux_err,A3_sum_flux_err,A3_mean_flux_err,A3_median_flux_err,A3_min_flux,A3_max_flux,A3_std_flux,A3_sum_flux,A3_mean_flux,A3_median_flux,A3_count_detected,A3_std_detected,A3_sum_detected,A3_mean_detected,A3_median_detected,A4_min_flux_err,A4_max_flux_err,A4_std_flux_err,A4_sum_flux_err,A4_mean_flux_err,A4_median_flux_err,A4_min_flux,A4_max_flux,A4_std_flux,A4_sum_flux,A4_mean_flux,A4_median_flux,A4_count_detected,A4_std_detected,A4_sum_detected,A4_mean_detected,A4_median_detected,A5_min_flux_err,A5_max_flux_err,A5_std_flux_err,A5_sum_flux_err,A5_mean_flux_err,A5_median_flux_err,A5_min_flux,A5_max_flux,A5_std_flux,A5_sum_flux,A5_mean_flux,A5_median_flux,A5_count_detected,A5_std_detected,A5_sum_detected,A5_mean_detected,A5_median_detected,p0_region_minus_4,p0_region_minus_3,p0_region_minus_2,p0_region_minus_1,p0_region_plus_1,p0_region_plus_2,p0_region_plus_3,p0_region_plus_4,p1_region_minus_4,p1_region_minus_3,p1_region_minus_2,p1_region_minus_1,p1_region_plus_1,p1_region_plus_2,p1_region_plus_3,p1_region_plus_4,p2_region_minus_4,p2_region_minus_3,p2_region_minus_2,p2_region_minus_1,p2_region_plus_1,p2_region_plus_2,p2_region_plus_3,p2_region_plus_4,p3_region_minus_4,p3_region_minus_3,p3_region_minus_2,p3_region_minus_1,p3_region_plus_1,p3_region_plus_2,p3_region_plus_3,p3_region_plus_4,p4_region_minus_4,p4_region_minus_3,p4_region_minus_2,p4_region_minus_1,p4_region_plus_1,p4_region_plus_2,p4_region_plus_3,p4_region_plus_4,p5_region_minus_4,p5_region_minus_3,p5_region_minus_2,p5_region_minus_1,p5_region_plus_1,p5_region_plus_2,p5_region_plus_3,p5_region_plus_4,number_of_0s,number_of_1s,number_of_2s,number_of_3s,number_of_4s,number_of_5s,percent_p0_region_minus_4,percent_p0_region_minus_3,percent_p0_region_minus_2,percent_p0_region_minus_1,percent_p0_region_plus_1,percent_p0_region_plus_2,percent_p0_region_plus_3,percent_p0_region_plus_4,percent_p1_region_minus_4,percent_p1_region_minus_3,percent_p1_region_minus_2,percent_p1_region_minus_1,percent_p1_region_plus_1,percent_p1_region_plus_2,percent_p1_region_plus_3,percen

In [23]:
classes = sorted(y.unique())

# Taken from Giba's topic : https://www.kaggle.com/titericz
# https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
# with Kyle Boone's post https://www.kaggle.com/kyleboone
class_weight = {
    c: 1 for c in classes
}
for c in [64, 15]:
    class_weight[c] = 2

print('Unique classes : ', classes)

train_id = temp['object_id']
del temp['object_id']

Unique classes :  [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]


In [24]:
# Compute weights
w = y.value_counts()
weights = {i : np.sum(w) / w[i] for i in w.index}

def multi_weighted_logloss(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds
    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return loss


def lgb_multi_weighted_logloss(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds.reshape(y_true.shape[0], len(classes), order='F')

    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return 'wloss', loss, False

In [25]:
%%time
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
clfs = []
importances = pd.DataFrame()
lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 14,
    'metric': 'multi_logloss',
    'learning_rate': 0.03,
    'subsample': .9,
    'colsample_bytree': 0.5,
    'reg_alpha': .01,
    'reg_lambda': .01,
    'min_split_gain': 0.01,
    'min_child_weight': 10,
    'n_estimators': 1000,
    'silent': -1,
    'verbose': -1,
    'max_depth': 3
}

CPU times: user 328 µs, sys: 22 µs, total: 350 µs
Wall time: 354 µs


In [26]:
# Compute weights
w = y.value_counts()
weights = {i : np.sum(w) / w[i] for i in w.index}

oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))

In [27]:
%%time
for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
    trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
    val_x, val_y = temp.iloc[val_], y.iloc[val_]

    clf = lgb.LGBMClassifier(**lgb_params)
    clf.fit(
        trn_x, trn_y,
        eval_set=[(trn_x, trn_y), (val_x, val_y)],
        eval_metric=lgb_multi_weighted_logloss,
        verbose=100,
        early_stopping_rounds=50,
        sample_weight=trn_y.map(weights)
    )
    oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
    print(multi_weighted_logloss(val_y, oof_preds[val_, :]))

    imp_df = pd.DataFrame()
    imp_df['feature'] = temp.columns
    imp_df['gain'] = clf.feature_importances_
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)

    clfs.append(clf)

print('MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))

Training until validation scores don't improve for 50 rounds.
[100]	training's multi_logloss: 0.775736	training's wloss: 0.763268	valid_1's multi_logloss: 1.13369	valid_1's wloss: 0.932766
[200]	training's multi_logloss: 0.51069	training's wloss: 0.496706	valid_1's multi_logloss: 0.902439	valid_1's wloss: 0.750107
[300]	training's multi_logloss: 0.392438	training's wloss: 0.378183	valid_1's multi_logloss: 0.812506	valid_1's wloss: 0.705268
[400]	training's multi_logloss: 0.323172	training's wloss: 0.30938	valid_1's multi_logloss: 0.766819	valid_1's wloss: 0.695019
Early stopping, best iteration is:
[395]	training's multi_logloss: 0.325946	training's wloss: 0.312081	valid_1's multi_logloss: 0.768242	valid_1's wloss: 0.69412
0.6941201982291848
Training until validation scores don't improve for 50 rounds.
[100]	training's multi_logloss: 0.778453	training's wloss: 0.767101	valid_1's multi_logloss: 1.14984	valid_1's wloss: 0.931817
[200]	training's multi_logloss: 0.508975	training's wloss: 

In [28]:
print(list(temp.columns.values)[:5])

['flux_min', 'flux_max', 'flux_mean', 'flux_median', 'flux_std']


In [29]:
print(list(test_metadata_kaggle.columns.values)[:5])

['object_id', 'flux_min', 'flux_max', 'flux_mean', 'flux_median']


In [30]:
imp_df.sort_values(by ='gain',ascending=False).reset_index(drop=True).head(20)

,feature,gain,fold
0,hostgal_photoz,3016,5
1,mjd_diff_det,1622,5
2,hostgal_photoz_err,1208,5
3,flux_by_flux_ratio_sq_skew,736,5
4,flux_dif2,708,5
5,flux_skew,668,5
6,flux_dif3,443,5
7,"5__fft_coefficient__coeff_1__attr_""abs""",430,5
8,2__skewness,408,5
9,A2_median_flux,389,5


In [31]:
%%time
temp_test = test_metadata_kaggle.merge(right = test_metadata[[x for x in test_metadata.columns if x not in ignored_columns ]] ,how='left',on = 'object_id')

CPU times: user 7.11 s, sys: 8.2 s, total: 15.3 s
Wall time: 16.1 s


In [32]:
del temp_test['object_id']

In [33]:
print(temp.shape,temp_test.shape)

(7848, 259) (3492890, 259)


In [34]:
list(temp.columns) == list(temp_test.columns)

True

In [35]:
del test_metadata,test_metadata_kaggle
gc.collect()

662

In [36]:
%%time
test_pred0 = pd.DataFrame()
test_pred1 = pd.DataFrame()
test_pred2 = pd.DataFrame()
test_pred3 = pd.DataFrame()
test_pred4 = pd.DataFrame()

list_of_df = [test_pred0,test_pred1,test_pred2,test_pred3,test_pred4]

for num,c in enumerate(clfs):
    print(num)
    for k in range(0,len(temp_test),500000):
        print(k)
        test_pred = pd.DataFrame(c.predict_proba(temp_test[ k:k+500000] ))
        list_of_df[num] = pd.concat([list_of_df[num],test_pred],axis=0)
        del test_pred

0
0
500000
1000000
1500000
2000000
2500000
3000000
1
0
500000
1000000
1500000
2000000
2500000
3000000
2
0
500000
1000000
1500000
2000000
2500000
3000000
3
0
500000
1000000
1500000
2000000
2500000
3000000
4
0
500000
1000000
1500000
2000000
2500000
3000000
CPU times: user 2h 27min 6s, sys: 51.2 s, total: 2h 27min 57s
Wall time: 22min 41s


In [37]:
list_of_df[0].head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.000281,0.004422,0.000270,0.344548,0.347692,0.000263,0.116757,0.000412,0.000369,0.005354,0.000506,0.174233,0.000342,0.004551
1,0.000127,0.008651,0.000206,0.085461,0.058082,0.000129,0.080257,0.000630,0.000568,0.073903,0.002124,0.681731,0.000139,0.007992
2,0.000162,0.009019,0.000483,0.095563,0.081258,0.000163,0.052025,0.001311,0.000660,0.129288,0.007680,0.589638,0.000167,0.032585
3,0.000126,0.005254,0.000182,0.069050,0.043812,0.000127,0.084351,0.000809,0.000400,0.216663,0.000905,0.557446,0.000140,0.020734
4,0.000146,0.002963,0.000134,0.091677,0.206068,0.000129,0.058246,0.000208,0.000232,0.049655,0.000162,0.589259,0.000128,0.000992


In [39]:
list_of_df[1].head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.000182,0.003368,0.000161,0.351556,0.419954,0.000149,0.051319,0.000194,0.000202,0.002981,0.000240,0.166129,0.000162,0.003404
1,0.000175,0.017510,0.000295,0.118819,0.056489,0.000162,0.146501,0.003275,0.001091,0.043694,0.000688,0.596244,0.000173,0.014883
2,0.000093,0.006905,0.000198,0.093250,0.070387,0.000085,0.047511,0.002897,0.000446,0.199551,0.003536,0.555574,0.000090,0.019477
3,0.000104,0.002632,0.000171,0.081121,0.032046,0.000097,0.070537,0.005313,0.000425,0.413086,0.000543,0.377657,0.000099,0.016168
4,0.000120,0.002932,0.000094,0.083905,0.252335,0.000090,0.052518,0.000131,0.000177,0.026225,0.000104,0.580343,0.000089,0.000938


In [40]:
test_pred = pd.DataFrame()
test_pred = (list_of_df[0] + list_of_df[1] + list_of_df[2] + list_of_df[3] + list_of_df[4])/5

In [41]:
test_pred.shape

(3492890, 14)

In [42]:
test_pred.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.000231,0.002695,0.000227,0.367970,0.343678,0.000221,0.109954,0.000284,0.000283,0.003941,0.000311,0.166403,0.000263,0.003538
1,0.000196,0.009580,0.000323,0.104193,0.054497,0.000197,0.091343,0.001509,0.000809,0.090307,0.000958,0.634474,0.000211,0.011403
2,0.000158,0.006373,0.000412,0.078465,0.072209,0.000159,0.047165,0.001841,0.000562,0.149575,0.003235,0.618613,0.000161,0.021074
3,0.000144,0.003167,0.000223,0.069961,0.033894,0.000144,0.077446,0.003009,0.000415,0.293136,0.000590,0.495949,0.000155,0.021767
4,0.000151,0.003247,0.000143,0.083528,0.202193,0.000139,0.045759,0.000188,0.000240,0.040655,0.000156,0.622550,0.000137,0.000913


In [43]:
test_pred[14] = 0.18

In [44]:
test_pred = test_pred / 1.18

In [45]:
test_pred.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.000196,0.002284,0.000192,0.311839,0.291253,0.000187,0.093182,0.000241,0.000240,0.003340,0.000263,0.141019,0.000223,0.002999,0.152542
1,0.000166,0.008119,0.000274,0.088299,0.046184,0.000167,0.077409,0.001279,0.000685,0.076532,0.000812,0.537690,0.000179,0.009664,0.152542
2,0.000134,0.005400,0.000349,0.066496,0.061194,0.000135,0.039970,0.001560,0.000476,0.126758,0.002741,0.524248,0.000137,0.017859,0.152542
3,0.000122,0.002684,0.000189,0.059289,0.028724,0.000122,0.065632,0.002550,0.000352,0.248420,0.000500,0.420296,0.000132,0.018447,0.152542
4,0.000128,0.002751,0.000122,0.070786,0.171350,0.000118,0.038779,0.000160,0.000204,0.034454,0.000132,0.527585,0.000116,0.000774,0.152542


In [46]:
temp_columns = ['object_id','class_6','class_15','class_16','class_42','class_52','class_53','class_62','class_64','class_65','class_67','class_88','class_90','class_92','class_95','class_99']

In [47]:
test_pred.columns = temp_columns[1:]

In [48]:
test_pred['object_id'] = test_id

In [49]:
test_pred = test_pred[temp_columns]

In [54]:
test_pred.head()

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
0,13,0.000196,0.002283,0.000192,0.311768,0.291260,0.000187,0.093201,0.000241,0.000240,0.003340,0.000263,0.140991,0.000223,0.002998,0.152588
1,14,0.000166,0.008118,0.000274,0.088318,0.046173,0.000167,0.077393,0.001279,0.000685,0.076538,0.000812,0.537598,0.000179,0.009666,0.152588
2,17,0.000134,0.005402,0.000350,0.066467,0.061188,0.000135,0.039978,0.001560,0.000476,0.126709,0.002741,0.524414,0.000136,0.017853,0.152588
3,23,0.000122,0.002684,0.000189,0.059296,0.028717,0.000122,0.065613,0.002550,0.000352,0.248413,0.000500,0.420410,0.000132,0.018448,0.152588
4,34,0.000128,0.002752,0.000122,0.070801,0.171387,0.000118,0.038788,0.000160,0.000203,0.034454,0.000132,0.527344,0.000116,0.000774,0.152588


In [53]:
test_pred.shape

(3492890, 16)

In [55]:
%%time
test_pred.to_csv('test_pred_5.csv',index=False)

CPU times: user 49.9 s, sys: 553 ms, total: 50.5 s
Wall time: 50.6 s
